<a href="https://colab.research.google.com/github/paveethranswam/Speech-style-transfer-and-ASR/blob/main/Wav2Vec2_fine_tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers  datasets librosa jiwer

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
!apt-get install libsox-fmt-all libsox-dev sox > /dev/null
!python -m pip install torchaudio > /dev/null
!python -m pip install git+https://github.com/facebookresearch/WavAugment.git > /dev/null
!git clone --recursive https://github.com/parlance/ctcdecode.git
!cd ctcdecode && pip install .

  Running command git clone -q https://github.com/facebookresearch/WavAugment.git /tmp/pip-req-build-w_4fv9kr
fatal: destination path 'ctcdecode' already exists and is not an empty directory.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Processing /content/ctcdecode
  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this functionality. You can find discussion regarding this at https://github.com/pypa/pip/issues/7555.
  Created wheel for ctcdecode: filename=ctcdecode-1.0.3-cp38-cp38-linux_x86_64.whl size=13476308 sha256=a95ab94aae0aec4d93a08db163ea2894c55a287b13acbf97f966afa02784e557
  Stored in directory: /tmp/pip-ephem-wheel-cache-j5g7og_h/wheels/1f/4a/18/d4116614c3e7ef50f26cb105531a733728c0adf

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
from datasets import load_dataset, load_metric,Audio
import transformers
import random
import numpy as np
import pandas as pd
import re
from IPython.display import display, HTML
from transformers import AutoProcessor
import librosa
import torch
from datasets import ClassLabel
from transformers import Wav2Vec2CTCTokenizer
from transformers import Wav2Vec2FeatureExtractor
from transformers import Wav2Vec2Processor
from transformers import Wav2Vec2ForCTC
from torch.optim import AdamW
from torch.optim.lr_scheduler import LinearLR
from transformers import get_scheduler
import augment
import json
from jiwer import wer
import os
import tqdm
import ctcdecode
from functools import partial
from IPython.display import display, HTML

In [5]:
# Get device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"Using {device} device")

Using cuda:0 device


In [6]:
minds = load_dataset("PolyAI/minds14", name="en-US", split="train[:100]")

In [7]:
minds = minds.remove_columns(["english_transcription", "intent_class", "lang_id"])

In [8]:
minds

Dataset({
    features: ['path', 'audio', 'transcription'],
    num_rows: 100
})

In [9]:
def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    display(HTML(df.to_html()))

In [10]:
show_random_elements(minds.remove_columns(["path", "audio"]))

,transcription
0,teach me how to pay bill online
1,I was wondering if I was able to set up a joint account through the app
2,I would like to set up a joint account can I do that in the app
3,I would like to be showing my account balance please
4,show me my account balance please
5,yes I'd like to set up a joint account I'm allowed to anyone
6,I'd like to pay a bail
7,I was wanting to set up a joint account with my spouse and I was wondering do we each get separate card debit card and separate way of knowing who is accessing the thank you
8,hi I want you to tell me my account balance until I'm just see it and I need to be able to reconcile a my account
9,I'm on your ass and my husband and I are trying to set up a joint account I can see where I set up an account by myself but I'm not sure where to go to set up a joint account can you help me with this please thank you


In [11]:
chars_to_ignore_regex = '[\,\?\.\!\-\;\:\"\“\%\‘\”\�]'

def remove_special_characters(batch):
    batch["transcription"] = re.sub(chars_to_ignore_regex, '', batch["transcription"]).upper()
    return batch

minds = minds.map(remove_special_characters)

In [12]:
vocab_dict = {'vocab': [],'all_text': []}
def extract_all_chars(batch):
  global vocab_dict
  all_text = " ".join(batch["transcription"])
  vocab_list = list(set(all_text))
  extras = ["", "", "", ""]
  vocab = {v: k for k, v in enumerate(extras)}
  for k, v in enumerate(vocab_list):
    if v == " ":
      vocab["|"] = k+len(extras)
    else:
      vocab[v] = k+len(extras)
  vocab_dict['vocab'].extend(vocab.keys())
  vocab_dict['all_text'].append(all_text)
  return vocab_dict

m1 = minds.remove_columns(["path", "audio"])
vocabs = m1.map(extract_all_chars)

In [13]:
vocab_dict['vocab'] = list(set(vocab_dict['vocab']))

In [14]:
print(vocab_dict)

{'vocab': [], 'all_text': []}


In [15]:
processor = AutoProcessor.from_pretrained("facebook/wav2vec2-base")

/usr/local/lib/python3.8/dist-packages/transformers/configuration_utils.py:369: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(


In [12]:
minds = minds.cast_column("audio", Audio(sampling_rate=16_000))

In [13]:
minds = minds.train_test_split(test_size=0.2)

In [18]:
minds['train']['audio'][0]['array']

array([-6.5018248e-06, -3.7291844e-05,  7.7234308e-06, ...,
       -1.2312034e-04, -7.7547038e-06,  3.3684842e-05], dtype=float32)

In [14]:
class SpeechTransform:
  def __init__(self, p=0.5):
    self.p = p

  def __call__(self, y):
    if np.random.rand() < self.p:
      return self.apply(y)
    return y

  def apply(self, y: np.ndarray):
      raise NotImplementedError

In [15]:
class ClipEffect(SpeechTransform):
  def __init__(self, p=0.5, sr=16000, rate=0.25):
    super().__init__(p)
    self.rate = rate
    self.sr = sr
  
  def apply(self, x):
    clip_chain = augment.EffectChain().clip(self.rate)
    return clip_chain.apply(x, src_info={'rate': self.sr})

In [16]:
class PitchShift(SpeechTransform):
  def __init__(self, p=0.5, sr=16000, max_rate=200):
    super().__init__(p)
    self.sr = sr
    self.max_rate = max_rate

  def apply(self, x):
    rate = np.random.randint(-self.max_rate, self.max_rate)
    return augment.EffectChain().pitch(rate).rate(self.sr).apply(x, src_info={'rate': self.sr})

In [17]:
class ReverbEffect(SpeechTransform):
  def __init__(self, p=0.5, sr=16000, max_rate=101):
    super().__init__(p)
    self.sr = sr
    self.max_rate = max_rate
  
  def apply(self, x):
    rnd_rate = np.random.randint(0, self.max_rate)
    return augment.EffectChain().reverb(50, 50, rnd_rate).channels(1).apply(x, src_info={'rate': self.sr})

In [18]:
class TimeDropout(SpeechTransform):
  def __init__(self, p=0.5, sr=16000, max_seconds=0.5):
    super().__init__(p)
    self.sr = sr
    self.max_seconds = max_seconds
  
  def apply(self, x):
    return augment.EffectChain().time_dropout(max_seconds=self.max_seconds).apply(x, src_info={'rate': self.sr})

In [19]:
class TimeShift(SpeechTransform):
  def __init__(self, p=0.5, sr=16000, min_shift=0.7, max_shift=2):
    super().__init__(p)
    self.sr = sr
    self.max_shift = max_shift
    self.min_shift = min_shift
  
  def apply(self, x):
    x = x.numpy()
    rate = np.random.uniform(self.min_shift, self.max_shift)
    return torch.as_tensor(librosa.effects.time_stretch(x, rate))

In [20]:
class Compose:
  def __init__(self, transforms):
    self.transforms = transforms
  def __call__(self, x):
    for tr in self.transforms:
      y = tr(x)
    return y

In [21]:
aug_transforms = Compose([ClipEffect(p=0.6),
                          PitchShift(p=0.6),
                          ReverbEffect(p=0.6),
                          TimeDropout(p=0.6),
                          TimeShift(p=0.8)])

In [22]:
minds

DatasetDict({
    train: Dataset({
        features: ['path', 'audio', 'transcription'],
        num_rows: 80
    })
    test: Dataset({
        features: ['path', 'audio', 'transcription'],
        num_rows: 20
    })
})

In [23]:
def apply_transforms(batch):
      audio = batch["audio"] 
      sp_arr = aug_transforms(torch.Tensor(audio['array']))
      batch['audio']['array'] = sp_arr
      return batch

minds = minds.map(apply_transforms)

  0%|          | 0/80 [00:00<?, ?ex/s]

  0%|          | 0/20 [00:00<?, ?ex/s]

In [30]:
vocab_list = vocab_dict['vocab']

vocab_list = {v: k for k, v in enumerate(vocab_list)}
vocab_list

vocab_list["|"] = len(vocab_list)
# del vocab_list['']
for key in vocab_list.keys():
  vocab_list[key] -= 1 
print(vocab_list)

{'|': -1}


In [31]:
label_dict = {"": 0, 
              "": 1, 
              "": 2,
              "": 3,
              "|": 4,
              "E": 5,
              "T": 6,
              "A": 7,
              "O": 8,
              "N": 9,
              "I": 10,
              "H": 11,
              "S": 12,
              "R": 13,
              "D": 14,
              "L": 15,
              "U": 16,
              "M": 17,
              "W": 18,
              "C": 19,
              "F": 20,
              "G": 21,
              "Y": 22,
              "P": 23,
              "B": 24,
              "V": 25,
              "K": 26,
              "'": 27,
              "X": 28,
              "J": 29,
              "Q": 30,
              "Z": 31
              }

label_dict['[UNK]'] = len(label_dict)
label_dict['[PAD]'] = len(label_dict)
print(len(label_dict))
labels = [key for key, value in label_dict.items()]

31


In [32]:
vocab_list['[UNK]'] = len(vocab_list)
vocab_list['[PAD]'] = len(vocab_list)
print(len(vocab_list))

3


In [33]:
with open('/content/drive/MyDrive/vocab.json', 'w') as vocab_file:
    json.dump(label_dict, vocab_file)

In [24]:
# tokenizer = Wav2Vec2CTCTokenizer("/content/drive/MyDrive/vocab.json", unk_token="[UNK]", pad_token="[PAD]", word_delimiter_token="|")
tokenizer = Wav2Vec2CTCTokenizer.from_pretrained("facebook/wav2vec2-large-960h-lv60-self")

In [25]:
feature_extractor = Wav2Vec2FeatureExtractor(feature_size=1, sampling_rate=16000, padding_value=0.0, do_normalize=True, return_attention_mask=False)

In [26]:
processor = Wav2Vec2Processor(feature_extractor=feature_extractor, tokenizer=tokenizer)

In [27]:
minds["train"]

Dataset({
    features: ['path', 'audio', 'transcription'],
    num_rows: 80
})

In [28]:
rand_int = random.randint(0, len(minds["train"]))

print("Target text:", minds["train"][rand_int]["transcription"])
print("Input array shape:", np.asarray(minds["train"][rand_int]["audio"]["array"]).shape)
print("Sampling rate:", minds["train"][rand_int]["audio"]["sampling_rate"])

Target text: HOW DO I CREATE A JOINT ACCOUNT
Input array shape: (26979,)
Sampling rate: 16000


In [29]:
def prepare_dataset(batch):
    audio = batch["audio"]

    # batched output is "un-batched" to ensure mapping is correct
    batch["input_values"] = processor(audio["array"], sampling_rate=audio["sampling_rate"]).input_values[0]
    
    with processor.as_target_processor():
        batch["labels"] = processor(batch["transcription"]).input_ids
    return batch

In [30]:
minds_tr = minds.map(prepare_dataset, remove_columns=minds.column_names["train"], num_proc=4)

#0:   0%|          | 0/20 [00:00<?, ?ex/s]

#2:   0%|          | 0/20 [00:00<?, ?ex/s]

#1:   0%|          | 0/20 [00:00<?, ?ex/s]

/usr/local/lib/python3.8/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:155: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(


#3:   0%|          | 0/20 [00:00<?, ?ex/s]

...

/usr/local/lib/python3.8/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:155: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:155: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(


#2:   0%|          | 0/5 [00:00<?, ?ex/s]

/usr/local/lib/python3.8/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:155: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(


#3:   0%|          | 0/5 [00:00<?, ?ex/s]

/usr/local/lib/python3.8/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:155: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(


In [31]:
def collate_function(batch, processor, padding=True, max_length=None, 
                     max_length_labels=None, pad_to_multiple_of=None, pad_to_multiple_of_labels=None):
    b_X = [{"input_values": sample["input_values"]} for sample in batch]
    b_Y = [{"input_ids": sample["labels"]} for sample in batch]
    features = processor.pad(b_X,
              padding=padding,
              max_length=max_length,
              pad_to_multiple_of=pad_to_multiple_of,
              return_tensors="pt")

    with processor.as_target_processor():
        batchY = processor.pad(b_Y,
          padding=padding,
          max_length=max_length_labels,
          pad_to_multiple_of=pad_to_multiple_of_labels,
          return_tensors="pt")
    labels = batchY["input_ids"].masked_fill(batchY.attention_mask.ne(1), -100)
    features["labels"] = labels
    return features

In [32]:
collate_fn = partial(collate_function, processor=processor, padding=True, max_length=None, 
                     max_length_labels=None, pad_to_multiple_of=None, pad_to_multiple_of_labels=None)

In [33]:
version = 2
model_save_path = "/content/drive/MyDrive/wav2vec_finetuning/"
model_name = f"wav2vec_{version}"
batch_size =3
epochs = 100
lr = 0.0002

In [34]:
train_dataloader = torch.utils.data.DataLoader(minds_tr["train"], batch_size=batch_size, shuffle=True, collate_fn=collate_fn)

In [46]:
batch_sample = next(iter(train_dataloader))

/usr/local/lib/python3.8/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:155: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(


In [47]:
print(batch_sample.keys())

dict_keys(['input_values', 'labels'])


In [48]:
print(batch_sample["input_values"].shape)
print(batch_sample["labels"].shape)

torch.Size([3, 143360])
torch.Size([3, 113])


In [49]:
batch_sample['labels'][0]

tensor([  10,    4,   18,    8,   16,   15,   14,    4,   15,   10,   26,    5,
           4,    6,    8,    4,   26,    9,    8,   18,    4,   18,   11,    7,
           6,    4,   17,   22,    4,   19,   16,   13,   13,    5,    9,    6,
           4,    7,   19,   19,    8,   16,    9,    6,    4,   24,    7,   15,
           7,    9,   19,    5,    4,   12,   11,    8,   18,   12, -100, -100,
        -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100,
        -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100,
        -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100,
        -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100,
        -100, -100, -100, -100, -100])

In [50]:
print(len(train_dataloader))

27


In [35]:
model = Wav2Vec2ForCTC.from_pretrained(
    "facebook/wav2vec2-large-960h-lv60-self",
    gradient_checkpointing=True,
    ctc_loss_reduction="mean",
    pad_token_id=processor.tokenizer.pad_token_id,
    vocab_size=len(processor.tokenizer),
    ignore_mismatched_sizes = True)

Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-large-960h-lv60-self and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [36]:
os.makedirs(model_save_path, exist_ok=True)

In [37]:
model.freeze_feature_encoder()

In [38]:
def compute_metrics(labels, preds):
    preds = torch.argmax(preds, axis=-1)
    labels[labels == -100] = processor.tokenizer.pad_token_id
    pred_str = processor.batch_decode(preds)
    label_str = processor.batch_decode(labels, group_tokens=False)
    return wer(label_str, pred_str)

In [39]:
optimizer = AdamW(model.parameters(), lr=lr)

In [40]:
num_training_steps = epochs*len(train_dataloader)

lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps
)

In [41]:
scaler = torch.cuda.amp.GradScaler()

In [42]:
model = model.to(device)
model = model.train()

In [59]:
torch.save(model, os.path.join(model_save_path, model_name+".pt"))
processor.save_pretrained(os.path.join(model_save_path, model_name+"_vocab"))

In [60]:
def train_step(train_dataloader, optimizer, lr_scheduler, processor, verbose_at=5, print_sentences=False):
    losses = []
    wers = []
    for step, data in enumerate(train_dataloader):
        optimizer.zero_grad()
        # move inputs to device
        batch = {k: v.to(device) for k, v in data.items()}
        with torch.cuda.amp.autocast():
          outputs = model(**batch)
        loss = outputs.loss
        losses.append(loss.cpu().detach().numpy())
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        lr_scheduler.step()
        preds = outputs.logits
        out_proba = torch.nn.functional.softmax(preds, dim=-1)
        labels = data["labels"]
        metrics = compute_metrics(labels, out_proba)
        wers.append(metrics)
        if step%verbose_at == 0:
            print(f"WER: {metrics} \t LOSS: {loss.cpu().detach().numpy()}")
            if print_sentences:
              for l, p in zip(labels, preds):
                  predicted_str = processor.tokenizer.decode(torch.argmax(p, dim =-1))
                  label_str = processor.tokenizer.decode(l)
                  print(f"True: {label_str}; Predicted: {predicted_str}")
    return {"loss": np.mean(losses), "wer":np.mean(wers)}

In [61]:
no_epochs = range(epochs)
epochs_progress_bar = tqdm.tqdm(no_epochs)
curr_best_loss = 1e10
for n in no_epochs:
    res = train_step(train_dataloader, optimizer, lr_scheduler, 
                     processor, verbose_at=len(train_dataloader)//2)
    res["best_loss"] = curr_best_loss
    if curr_best_loss > res["loss"]:
        print("Best model, saving at ",model_save_path)
        torch.save(model, os.path.join(model_save_path, model_name+".pt"))
        curr_best_loss = res["loss"]
        res["best_loss"] = curr_best_loss
    epochs_progress_bar.update(1)

  0%|          | 0/100 [00:00<?, ?it/s]/usr/local/lib/python3.8/dist-packages/torch/optim/lr_scheduler.py:138: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn("Detected call of `lr_scheduler.step()` before `optimizer.step()`. "


WER: 0.5918367346938775 	 LOSS: 1.606234073638916
WER: 0.9 	 LOSS: 2.4168477058410645
WER: 0.7272727272727273 	 LOSS: 0.8213731050491333
Best model, saving at  /content/drive/MyDrive/wav2vec_finetuning/


  1%|          | 1/100 [00:36<1:00:36, 36.73s/it]

WER: 1.0 	 LOSS: 1.8681119680404663
WER: 0.9333333333333333 	 LOSS: 2.782756805419922
WER: 0.8666666666666667 	 LOSS: 1.9665186405181885
Best model, saving at  /content/drive/MyDrive/wav2vec_finetuning/


  2%|▏         | 2/100 [01:02<49:27, 30.28s/it]  

...

WER: 0.0 	 LOSS: 0.0014242252800613642
WER: 0.15555555555555556 	 LOSS: 0.10584160685539246
WER: 0.0 	 LOSS: 0.002962514292448759


 99%|█████████▉| 99/100 [34:52<00:19, 19.20s/it]

WER: 0.06382978723404255 	 LOSS: 0.024223193526268005
WER: 0.0 	 LOSS: 0.0034777275286614895
WER: 0.03225806451612903 	 LOSS: 0.027628563344478607


100%|██████████| 100/100 [35:11<00:00, 19.14s/it]

WER: 0.09090909090909091 	 LOSS: 0.10918695479631424


In [43]:
model_path = os.path.join(model_save_path, model_name+".pt")
pipeline_path= os.path.join(model_save_path, model_name+"_vocab")

In [44]:
model = torch.load(model_path)

In [45]:
model = model.eval()

In [46]:
minds = minds.map(apply_transforms, remove_columns=minds.column_names["test"])
minds_te = minds.map(prepare_dataset, remove_columns=minds.column_names["test"], num_proc=4)

  0%|          | 0/80 [00:00<?, ?ex/s]

  0%|          | 0/20 [00:00<?, ?ex/s]

In [47]:
print(minds_te)

DatasetDict({
    train: Dataset({
        features: [],
        num_rows: 0
    })
    test: Dataset({
        features: [],
        num_rows: 0
    })
})


In [48]:
class CTCBeamDecoder:
    def __init__(self, labels, blank_id=0, beam_size=100, kenlm_path=None):
        print("loading beam search with kenlm...")
        self.labels = labels

        # model_path = is the path to your external kenlm language model(LM). Default is none.
        # alpha = Weighting associated with the LMs probabilities. A weight of 0 means the LM has no effect.
        # beta = Weight associated with the number of words within our beam.

        self.ctcdecoder = ctcdecode.CTCBeamDecoder(
            self.labels, model_path=kenlm_path,
            alpha=0.6, beta=1,
            beam_width=beam_size, blank_id=blank_id)
        print("loading finished")

    def __call__(self, output, num_sentences=1):
        sentences = []
        for num in range(num_sentences):
          beam_result, beam_scores, timesteps, out_seq_len = self.ctcdecoder.decode(output)
          # beam_result[0][0][:out_seq_len[0][0]] get the top beam for the first item in batch
          sentences.append(self.output(beam_result[0][num], self.labels, out_seq_len[0][num]))
        return sentences

    def output(self, tokens, vocab, seq_len):
        out = ''.join([vocab[x] for x in tokens[0:seq_len]])
        # wave2vec implementation use | for space in vocabulary
        return out.replace("|", " ")

In [50]:
decode_and_rescore = CTCBeamDecoder(kenlm_path=None, 
                                          labels=labels, 
                                          blank_id=label_dict.get(""), 
                                          beam_size=100)

loading beam search with kenlm...
loading finished


In [49]:
test_dataloader = torch.utils.data.DataLoader(minds_tr["test"], batch_size=batch_size, shuffle=True, collate_fn=collate_fn)

In [50]:
for step, data in enumerate(test_dataloader):
            batch = {k: v.to(device) for k, v in data.items()}
            with torch.cuda.amp.autocast():
              outputs = model(**batch)
              logits = outputs.logits
              out_proba = torch.nn.functional.softmax(logits, dim=-1)
              predicted_ids = torch.argmax(logits, dim =-1)
              transcriptions = tokenizer.decode(predicted_ids[0])
              print(transcriptions)
              print(tokenizer.decode(data['labels'][0]))

/usr/local/lib/python3.8/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:155: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(


HOW DO I SET UP A JOINT ACCOUNT
HOW DO I SET UP A JOINT ACOUNT<unk>
HI I WAS TRYING TO SET UP A JOINT ACCOUNT IS THERE A SOMEWHERE ON THE APP THAT I CAN DO THAT OR SO IAVE TO DO THAT THROUGH YOU
HI I WAS TRYING TO SET UP A JOINT ACOUNT IS THERE A SOMEWHERE ON THE AP THAT I CAN DO THAT OR DO I HAVE TO DO THAT THROUGH YOU
A HELP TO SET UP A JOINT ACCOUNT WITH MY LIFEA THER THE
HENRY COUNTY SET UP A JOINT ACOUNT WITH MY WIFE AND WHERE ARE THEY AT<unk>
OPEN UP A JOINT ACCOUNT WITH MY PARTNER
OPEN UP A JOINT ACOUNT WITH MY PARTNER<unk>
SHOW ME MY ACCOUNT BALANCE PLEASE
SHOW ME MY ACOUNT BALANCE PLEASE<unk>
SHOW ME MY CURRENT ACCOUNT BALANCE
SHOW ME MY CURENT ACOUNT BALANCE<unk>
I NEED TO FIND OUT HOWB LY SET UP A JOINT ACCOUNT
I NED TO FIND OUT IF I PROBABLY SET UP A JOINT ACOUNT


In [ ]:
# next(iter(test_dataloader))['input_values']